In [1]:
import pandas as pd
import numpy as np
import os
import pickle
from recomendation_model.base_model import LightGBMRanker

os.chdir("c://Users//gufer//OneDrive//Documentos//FIAP//Fase_05//ML_Engineer_Datathon/")

In [3]:
# Carrega os dados de treino a partir dos arquivos Parquet
X_train = pd.read_parquet("data/train/X_train.parquet")
y_train = pd.read_parquet("data/train/y_train.parquet")
X_train_full = pd.read_parquet("data/train/X_train_full.parquet")

In [8]:
X_train.columns

Index(['isWeekend', 'relLocalState', 'relLocalRegion', 'relThemeMain',
       'relThemeSub', 'userTypeFreq', 'dayPeriodFreq', 'localStateFreq',
       'localRegionFreq', 'themeMainFreq', 'themeSubFreq'],
      dtype='object')

In [4]:
X_train_full.columns

Index(['userId', 'pageId', 'userType', 'isWeekend', 'dayPeriod',
       'issuedDatetime', 'timestampHistoryDatetime', 'coldStart', 'localState',
       'localRegion', 'themeMain', 'themeSub', 'relLocalState',
       'relLocalRegion', 'relThemeMain', 'relThemeSub', 'userTypeFreq',
       'dayPeriodFreq', 'localStateFreq', 'localRegionFreq', 'themeMainFreq',
       'themeSubFreq'],
      dtype='object')

In [4]:
y_train

,TARGET
0,25.220086
1,5.110230
2,0.962531
3,5.564775
4,3.244410
...,...
1556855,0.227597
1556856,25.220086
1556857,2.785324
1556858,-0.230356


In [5]:
group_train = pd.read_parquet("data/train/group_train.parquet")

# IMPORTANTE: Colocar essa extração em algum canto no pipeline
#group_train = group_train["groupCount"].values
group_train.describe()

,groupCount
count,10426.000000
mean,149.324765
std,6541.612217
min,1.000000
25%,1.000000
50%,2.000000
75%,7.000000
max,639130.000000


In [6]:
# # Carrega os dados de treino a partir dos arquivos Parquet
# X_train = pd.read_parquet("data/train/X_train.parquet").values
# y_train = pd.read_parquet("data/train/y_train.parquet").values.ravel()

In [7]:
# Defina um limite para o número máximo de interações por usuário
MAX_INTERACTIONS = 10000

def subsample_groups(df, group_col="userId", max_interactions=MAX_INTERACTIONS):
    """
    Para cada usuário, se o número de interações for maior que max_interactions,
    amostra aleatoriamente max_interactions linhas. Caso contrário, mantém todas as linhas.
    """
    return df.groupby(group_col, group_keys=False).apply(
        lambda x: x.sample(n=min(len(x), max_interactions), random_state=42)
    )

# Supondo que X_train seja o DataFrame com as features e que y_train contenha o TARGET
# Subamostra os dados de treino
X_train_sub = subsample_groups(X_train, group_col="userId", max_interactions=MAX_INTERACTIONS)
# Ajusta y_train para manter o mesmo índice
y_train_sub = y_train.loc[X_train_sub.index]

# Recalcule o vetor de grupos a partir dos dados amostrados
group_train = X_train_sub.groupby("userId").size().reset_index(name="groupCount")

# Verifique o resumo dos grupos
print(group_train.describe())

# Agora, treine o modelo utilizando os dados amostrados
# Note que para o parâmetro 'group' do LightGBM, passamos os valores da coluna 'groupCount'
model = LightGBMRanker()
model.train(X_train_sub.values, y_train_sub.values.ravel(), group_train["groupCount"].values)

# Salve o modelo em um arquivo pickle
import pickle
with open("lightgbm_ranker.pkl", "wb") as f:
    pickle.dump(model, f)

print("Modelo treinado e salvo em 'lightgbm_ranker.pkl'")

KeyError: 'userId'